In [1]:
import os
import torch
import scipy.stats as stats
import numpy as np

# CODE FILES HERE
from models.vae.vae import Encoder, Decoder, Vae, MODEL_NAME
from solver import Solver
from directories import Directories
from dataloader import DataLoader
from plots import plot_losses, plot_gaussian_distributions, plot_rl_kl, plot_latent_space, \
plot_latent_manifold, plot_faces_grid, plot_faces_samples_grid

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cpu



In [3]:
# Choose the dataset and tune hyperparameters here!
dataset = "FF"

batch_size = 128
optimizer = torch.optim.Adam

if dataset == "MNIST":
    epochs = 20
    hidden_dim = 500
    z_dim = 2
    beta = 4 if z_dim == 2 else 4
    batch_norm = False
    lr_scheduler = torch.optim.lr_scheduler.StepLR
    step_config = {
        "step_size" : 300,
        "gamma" : 0.1 # or 0.75
    }
    optim_config = {
        "lr": 1e-3,
        "weight_decay": None
    }
elif dataset == "LFW":
    epochs = 30
    hidden_dim = 700
    z_dim = 40
    beta = 1 if z_dim == 2 else 1
    batch_norm = True
    lr_scheduler = torch.optim.lr_scheduler.StepLR
    step_config = {
        "step_size" : 30,
        "gamma" : 0.1
    }
    optim_config = {
        "lr": 1e-2,
        "weight_decay": None
    }
elif dataset == "FF":
    epochs = 1500
    hidden_dim = 200
    z_dim = 2
    beta = 1
    batch_norm = True
    lr_scheduler = torch.optim.lr_scheduler.StepLR
    step_config = {
        "step_size" : 300,
        "gamma" : 0.1
    }
    optim_config = {
        "lr": 1e-2,
        "weight_decay": None
    }

In [4]:
directories = Directories(MODEL_NAME, dataset, z_dim)
data_loader = DataLoader(directories, batch_size, dataset)
model = Vae(data_loader.input_dim, hidden_dim, z_dim, batch_norm)
solver = Solver(model, data_loader, optimizer, z_dim, epochs, beta, step_config, optim_config, lr_scheduler=lr_scheduler)
solver.main()

+++++ START RUN | saved files in vae/FF_z=2_8 +++++
====> Epoch: 1 train set loss avg: 398.8467
====> Test set loss avg: 518.7070
0.80 seconds for epoch 1
====> Epoch: 2 train set loss avg: 382.9495
====> Test set loss avg: 390.6739
0.50 seconds for epoch 2
====> Epoch: 3 train set loss avg: 373.3598
====> Test set loss avg: 369.8354
0.49 seconds for epoch 3
====> Epoch: 4 train set loss avg: 366.8275
====> Test set loss avg: 362.6427
0.48 seconds for epoch 4
====> Epoch: 5 train set loss avg: 362.2196
====> Test set loss avg: 355.9145
0.48 seconds for epoch 5
====> Epoch: 6 train set loss avg: 358.8214
====> Test set loss avg: 352.2199
0.47 seconds for epoch 6
====> Epoch: 7 train set loss avg: 356.4108
====> Test set loss avg: 350.1155
0.49 seconds for epoch 7
====> Epoch: 8 train set loss avg: 354.6864
====> Test set loss avg: 348.0011
0.60 seconds for epoch 8
====> Epoch: 9 train set loss avg: 353.3590
====> Test set loss avg: 346.5825
0.51 seconds for epoch 9
====> Epoch: 10 train

====> Epoch: 79 train set loss avg: 347.3156
====> Test set loss avg: 346.0152
0.57 seconds for epoch 79
====> Epoch: 80 train set loss avg: 347.3455
====> Test set loss avg: 345.1803
0.63 seconds for epoch 80
====> Epoch: 81 train set loss avg: 347.3257
====> Test set loss avg: 344.3811
0.65 seconds for epoch 81
====> Epoch: 82 train set loss avg: 347.3947
====> Test set loss avg: 344.7350
0.53 seconds for epoch 82
====> Epoch: 83 train set loss avg: 347.3260
====> Test set loss avg: 347.6412
0.53 seconds for epoch 83
====> Epoch: 84 train set loss avg: 347.4352
====> Test set loss avg: 343.7434
0.55 seconds for epoch 84
====> Epoch: 85 train set loss avg: 347.3822
====> Test set loss avg: 344.3548
0.54 seconds for epoch 85
====> Epoch: 86 train set loss avg: 347.3971
====> Test set loss avg: 344.5550
0.53 seconds for epoch 86
====> Epoch: 87 train set loss avg: 347.3850
====> Test set loss avg: 344.9804
0.54 seconds for epoch 87
====> Epoch: 88 train set loss avg: 347.5168
====> Test

====> Epoch: 156 train set loss avg: 347.2388
====> Test set loss avg: 345.3148
0.63 seconds for epoch 156
====> Epoch: 157 train set loss avg: 347.2555
====> Test set loss avg: 345.9615
0.84 seconds for epoch 157
====> Epoch: 158 train set loss avg: 347.2526
====> Test set loss avg: 344.7004
0.75 seconds for epoch 158
====> Epoch: 159 train set loss avg: 347.1441
====> Test set loss avg: 346.6191
0.64 seconds for epoch 159
====> Epoch: 160 train set loss avg: 346.9707
====> Test set loss avg: 344.4359
0.75 seconds for epoch 160
====> Epoch: 161 train set loss avg: 347.1112
====> Test set loss avg: 346.1953
0.62 seconds for epoch 161
====> Epoch: 162 train set loss avg: 347.2965
====> Test set loss avg: 345.9316
0.63 seconds for epoch 162
====> Epoch: 163 train set loss avg: 347.2223
====> Test set loss avg: 345.2987
0.62 seconds for epoch 163
====> Epoch: 164 train set loss avg: 347.1356
====> Test set loss avg: 344.9417
0.63 seconds for epoch 164
====> Epoch: 165 train set loss avg: 

====> Epoch: 233 train set loss avg: 347.2057
====> Test set loss avg: 345.9751
0.74 seconds for epoch 233
====> Epoch: 234 train set loss avg: 347.0901
====> Test set loss avg: 346.4258
0.86 seconds for epoch 234
====> Epoch: 235 train set loss avg: 347.0907
====> Test set loss avg: 346.5020
0.79 seconds for epoch 235
====> Epoch: 236 train set loss avg: 347.1595
====> Test set loss avg: 346.3227
0.77 seconds for epoch 236
====> Epoch: 237 train set loss avg: 346.9691
====> Test set loss avg: 344.7961
0.73 seconds for epoch 237
====> Epoch: 238 train set loss avg: 347.0742
====> Test set loss avg: 345.2456
0.72 seconds for epoch 238
====> Epoch: 239 train set loss avg: 347.1862
====> Test set loss avg: 344.9291
0.72 seconds for epoch 239
====> Epoch: 240 train set loss avg: 347.1348
====> Test set loss avg: 345.3102
0.74 seconds for epoch 240
====> Epoch: 241 train set loss avg: 347.1241
====> Test set loss avg: 344.7534
0.74 seconds for epoch 241
====> Epoch: 242 train set loss avg: 

====> Epoch: 310 train set loss avg: 346.6792
====> Test set loss avg: 345.5456
0.84 seconds for epoch 310
====> Epoch: 311 train set loss avg: 346.6534
====> Test set loss avg: 345.6719
0.89 seconds for epoch 311
====> Epoch: 312 train set loss avg: 346.7506
====> Test set loss avg: 345.8716
0.87 seconds for epoch 312
====> Epoch: 313 train set loss avg: 346.9054
====> Test set loss avg: 345.9931
0.86 seconds for epoch 313
====> Epoch: 314 train set loss avg: 346.9145
====> Test set loss avg: 345.9256
0.87 seconds for epoch 314
====> Epoch: 315 train set loss avg: 346.7912
====> Test set loss avg: 345.7046
0.90 seconds for epoch 315
====> Epoch: 316 train set loss avg: 346.7750
====> Test set loss avg: 345.4841
0.83 seconds for epoch 316
====> Epoch: 317 train set loss avg: 346.7621
====> Test set loss avg: 345.5594
0.84 seconds for epoch 317
====> Epoch: 318 train set loss avg: 346.7957
====> Test set loss avg: 345.6527
0.83 seconds for epoch 318
====> Epoch: 319 train set loss avg: 

...

====> Epoch: 1155 train set loss avg: 346.7177
====> Test set loss avg: 346.2998
0.96 seconds for epoch 1155
====> Epoch: 1156 train set loss avg: 346.7046
====> Test set loss avg: 346.2952
0.94 seconds for epoch 1156
====> Epoch: 1157 train set loss avg: 346.6429
====> Test set loss avg: 346.4127
0.95 seconds for epoch 1157
====> Epoch: 1158 train set loss avg: 346.8859
====> Test set loss avg: 346.3960
0.95 seconds for epoch 1158
====> Epoch: 1159 train set loss avg: 346.6360
====> Test set loss avg: 346.3358
0.95 seconds for epoch 1159
====> Epoch: 1160 train set loss avg: 346.6446
====> Test set loss avg: 346.1394
0.95 seconds for epoch 1160
====> Epoch: 1161 train set loss avg: 346.7019
====> Test set loss avg: 346.3262
0.95 seconds for epoch 1161
====> Epoch: 1162 train set loss avg: 346.6598
====> Test set loss avg: 346.3318
0.96 seconds for epoch 1162
====> Epoch: 1163 train set loss avg: 346.5181
====> Test set loss avg: 346.1714
0.96 seconds for epoch 1163
====> Epoch: 1164 t

====> Epoch: 1231 train set loss avg: 346.6506
====> Test set loss avg: 346.4222
0.96 seconds for epoch 1231
====> Epoch: 1232 train set loss avg: 346.7350
====> Test set loss avg: 346.3962
0.97 seconds for epoch 1232
====> Epoch: 1233 train set loss avg: 346.7940
====> Test set loss avg: 346.2930
0.96 seconds for epoch 1233
====> Epoch: 1234 train set loss avg: 346.7004
====> Test set loss avg: 346.3628
0.97 seconds for epoch 1234
====> Epoch: 1235 train set loss avg: 346.7875
====> Test set loss avg: 346.2978
0.95 seconds for epoch 1235
====> Epoch: 1236 train set loss avg: 346.6480
====> Test set loss avg: 346.2862
0.96 seconds for epoch 1236
====> Epoch: 1237 train set loss avg: 346.5647
====> Test set loss avg: 346.3595
0.96 seconds for epoch 1237
====> Epoch: 1238 train set loss avg: 346.7049
====> Test set loss avg: 346.2385
0.96 seconds for epoch 1238
====> Epoch: 1239 train set loss avg: 346.6365
====> Test set loss avg: 346.3275
0.97 seconds for epoch 1239
====> Epoch: 1240 t

====> Epoch: 1307 train set loss avg: 346.6938
====> Test set loss avg: 346.2925
0.98 seconds for epoch 1307
====> Epoch: 1308 train set loss avg: 346.6982
====> Test set loss avg: 346.2983
0.97 seconds for epoch 1308
====> Epoch: 1309 train set loss avg: 346.5582
====> Test set loss avg: 346.4678
0.99 seconds for epoch 1309
====> Epoch: 1310 train set loss avg: 346.5239
====> Test set loss avg: 346.2714
0.99 seconds for epoch 1310
====> Epoch: 1311 train set loss avg: 346.6369
====> Test set loss avg: 346.3650
0.97 seconds for epoch 1311
====> Epoch: 1312 train set loss avg: 346.7885
====> Test set loss avg: 346.3553
0.98 seconds for epoch 1312
====> Epoch: 1313 train set loss avg: 346.6555
====> Test set loss avg: 346.2748
0.98 seconds for epoch 1313
====> Epoch: 1314 train set loss avg: 346.7552
====> Test set loss avg: 346.2819
0.98 seconds for epoch 1314
====> Epoch: 1315 train set loss avg: 346.5757
====> Test set loss avg: 346.3602
1.03 seconds for epoch 1315
====> Epoch: 1316 t

====> Epoch: 1383 train set loss avg: 346.6760
====> Test set loss avg: 346.1910
1.17 seconds for epoch 1383
====> Epoch: 1384 train set loss avg: 346.6220
====> Test set loss avg: 346.3395
1.13 seconds for epoch 1384
====> Epoch: 1385 train set loss avg: 346.7214
====> Test set loss avg: 346.4444
1.12 seconds for epoch 1385
====> Epoch: 1386 train set loss avg: 346.6497
====> Test set loss avg: 346.3228
1.18 seconds for epoch 1386
====> Epoch: 1387 train set loss avg: 346.7011
====> Test set loss avg: 346.3929
1.23 seconds for epoch 1387
====> Epoch: 1388 train set loss avg: 346.7557
====> Test set loss avg: 346.2857
1.15 seconds for epoch 1388
====> Epoch: 1389 train set loss avg: 346.6461
====> Test set loss avg: 346.2695
1.07 seconds for epoch 1389
====> Epoch: 1390 train set loss avg: 346.6702
====> Test set loss avg: 346.2541
1.25 seconds for epoch 1390
====> Epoch: 1391 train set loss avg: 346.7129
====> Test set loss avg: 346.3433
1.14 seconds for epoch 1391
====> Epoch: 1392 t

====> Epoch: 1459 train set loss avg: 346.6234
====> Test set loss avg: 346.4313
0.99 seconds for epoch 1459
====> Epoch: 1460 train set loss avg: 346.7386
====> Test set loss avg: 346.3679
1.04 seconds for epoch 1460
====> Epoch: 1461 train set loss avg: 346.5696
====> Test set loss avg: 346.4057
0.99 seconds for epoch 1461
====> Epoch: 1462 train set loss avg: 346.5456
====> Test set loss avg: 346.2527
0.98 seconds for epoch 1462
====> Epoch: 1463 train set loss avg: 346.6885
====> Test set loss avg: 346.4356
1.02 seconds for epoch 1463
====> Epoch: 1464 train set loss avg: 346.6576
====> Test set loss avg: 346.2875
0.99 seconds for epoch 1464
====> Epoch: 1465 train set loss avg: 346.7046
====> Test set loss avg: 346.2109
0.99 seconds for epoch 1465
====> Epoch: 1466 train set loss avg: 346.4751
====> Test set loss avg: 346.3108
0.98 seconds for epoch 1466
====> Epoch: 1467 train set loss avg: 346.7162
====> Test set loss avg: 346.2840
0.97 seconds for epoch 1467
====> Epoch: 1468 t

In [ ]:
# Insert name of model here if want to load a model, e.g. solver.save_model_dir + "/VAE_MNIST_train_loss=151.39_z=2.pt"
#solver = torch.load(solver.save_model_dir + "/VAE_MNIST_train_loss=97.15_z=20.pt")
#solver.model.eval()

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, solver.train_loss_history["train_loss_acc"], solver.test_loss_history)

In [ ]:
# Plotting the gaussian of z space and some metrics about the space
plot_gaussian_distributions(solver, len(solver.train_loss_history["train_loss_acc"]))

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 1
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
                             solver.train_loss_history["train_loss_acc"], \
                             solver.test_loss_history, \
                             solver.train_loss_history["recon_loss_acc"], \
                             solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}".format(
            epoch, train_loss, test_loss, rl, kl))
        print("overfitting: {:.2f}".format(abs(test_loss-train_loss)))
plot_rl_kl(solver, solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"])

In [ ]:
# visualize q(z) (latent space z)
if solver.z_dim == 2:
    if solver.data_loader.with_labels:
        plot_latent_space(solver, solver.z_space, var="z", title="classes", labels=solver.data_labels)
    else:
        plot_latent_space(solver, solver.z_space, var="z")
else:
    print("Plot of latent space not possible as dimension of z is not 2")

In [ ]:
# Visualizations of learned data manifold for generative models with two-dimensional latent space
if solver.z_dim == 2:
    if solver.data_loader.dataset == "MNIST":
        n = 20
        grid_x = grid_y = np.linspace(-3, 3, n)
        plot_latent_manifold(solver, "bone", grid_x, grid_y, n)
    if solver.data_loader.dataset == "LFW" or solver.data_loader.dataset == "FF":
        n = 10
        grid_x = grid_y = np.linspace(-3, 3, n)
        plot_latent_manifold(solver, "gray", grid_x, grid_y, n, fig_size=(10, 8))
else:
    print("Plot is not possible as dimension of z is not 2")

In [ ]:
# plots real faces and in grid samples
if solver.data_loader.dataset == "LFW" or solver.data_loader.dataset == "FF":
    n = 225
    n_cols = 15
    plot_faces_grid(n, n_cols, solver)
    plot_faces_samples_grid(n, n_cols, solver)

In [ ]:
last_train_loss = solver.train_loss_history["train_loss_acc"][-1]
torch.save(solver, solver.data_loader.directories.result_dir + "/model_VAE_" + solver.data_loader.dataset + "_train_loss=" + "{0:.2f}".format(last_train_loss) + "_z=" + str(solver.z_dim) + ".pt")